# 組み合わせの剰余演算

## 概要

組み合わせ $_n\mathrm{C}_k$ の計算では、$n$ の増加に伴い組み合わせの総数が非常に大きくなる。  
そのため、競技プログラミングでは組み合わせの剰余をとることがよくある。

## 組み合わせの計算

### 方法1

$n \leq 10^6$ 程度まで対応可能。

$$
\binom{n}{k} = \frac{n!}{k!(n-k)!} = n!(k!)^{-1}((n-k)!)^{-1}
$$

$n$ 以下の全ての $k$ に対して $k!, (k!)^{-1}$ の2種類の値を事前計算しておくことで、クエリ毎に $\mathcal{O}(1)$ で計算可能となる。  
階乗 $k!$ はforループで求めればよい。逆元計算は[合同](mod.ipynb)を参照。

In [ ]:
def binom_preprocess(n, MOD=10**9+7):
    f = [0 for i in range(n+1)]  # n!
    invf = [0 for i in range(n+1)]  # (n!)^-1
    f[0] = 1
    f[1] = 1
    invf[0] = 1
    invf[1] = 1
    for i in range(2, n+1):
        f[i] = f[i-1] * i % MOD
        invf[i] = pow(f[i], MOD-2, MOD)
    return f, invf

def binom(n, k, f, invf, MOD=10**9+7):
    if n < k or n < 0 or k < 0:
        return 0
    else:
        return (f[n] * invf[k] % MOD) * invf[n-k] % MOD

前処理の `pow(f[i], p-2, p)` を含むループはさらに計算量を落とすことができる。  
$a_k = (k!)^{-1}$ とおけば、漸化式 $a_{k-1} = k \cdot a_k$ が成り立つので、 $a_n = (n!)^{-1}$ だけ計算すれば他は漸化式から求まる。

計算量は事前計算に$\mathcal{O}(n)$、クエリ毎に$\mathcal{O}(1)$。

In [2]:
def binom_preprocess(n, MOD=10**9+7):
    f = [0 for i in range(n+1)]  # n!
    invf = [0 for i in range(n+1)]  # (n!)^-1
    f[0] = 1
    f[1] = 1
    invf[0] = 1
    invf[1] = 1
    for i in range(2, n+1):
        f[i] = f[i-1] * i % MOD
    invf[n] = pow(f[n], MOD-2, MOD)
    for i in range(n, 2, -1):
        invf[i-1] = invf[i] * i % MOD
    return f, invf

def binom(n, k, f, invf, MOD=10**9+7):
    if n < k or n < 0 or k < 0:
        return 0
    else:
        return (f[n] * invf[k] % MOD) * invf[n-k] % MOD

In [3]:
f, invf = binom_preprocess(10**6)
print(binom(10**6, 4, f, invf))

85121780


### 方法2

$\min(k, n-k) \leq 10^6$ 程度まで対応可能。 $n$ が大きくても対応できることが特徴。

$$
\binom{n}{k} = \frac{n!}{k!(n-k)!} = \frac{n(n-1) \ldots (n-k+1)}{k(k-1) \ldots 1}
$$

分母と分子を別々に計算し、それらの商をとる。  
分母、分子はいずれも $k$ 回のループで求まるので、全体の計算量はクエリ毎に $\mathcal{O}(k)$。

In [4]:
def binom(n, k, MOD=10**9+7):
    k = min(k, n-k)
    a = 1
    for i in range(n-k+1, n+1):
        a = a * i % MOD
    b = 1
    for i in range(1, k+1):
        b = b * i % MOD
    return a * pow(b, MOD-2, MOD) % MOD

In [5]:
print(binom(10**12, 10**6))

344082972


## 組み合わせの和の計算

### $n$ を固定したときの和

$$
\sum^{n}_{k=0} \binom{n}{k} x^k = (1+x)^n
$$

に対して $x=1$ を代入すると

$$
\begin{eqnarray}
\sum^{n}_{k=0} \binom{n}{k} 1^k &=& (1+1)^n \\
\sum^{n}_{k=0} \binom{n}{k} &=& 2^n \\
\end{eqnarray}
$$

となる。つまり、$n$ を固定したときの組み合わせの総数は $2^n$ である。

### $k$ を固定したときの和

組み合わせに関して $k=1$ または $k=2$ としたときの和は、和の公式や[三角数](https://ja.wikipedia.org/wiki/%E4%B8%89%E8%A7%92%E6%95%B0)と呼ばれる。  
一般に、次の公式が知られている。

$$
\begin{eqnarray}
1 + 2 + 3 + 4 + \ldots + \binom{n}{1} &=& \frac{n(n+1)}{2} \\
1 + 3 + 6 + 10 + \ldots + \binom{n}{2} &=& \frac{n(n+1)(n+2)}{6} \\
\end{eqnarray}
$$

任意の $k$ に対して和を求めるときは、個別に組み合わせを計算して足し合わせればよい。